# Extension to more qubits

## Define the Hamiltonian

In general, the game Hamiltonian should be represented on a graph.

In [1]:
import numpy as np
from functools import reduce
import math
from tqdm import tqdm
# from mpi4py import MPI  # Commented out - only needed for distributed grid search
from scipy.optimize import differential_evolution
import os
from jaxtyping import Float, Int, Bool, Complex
from numpy.typing import NDArray as Array

In [2]:
import networkx as nx
from typing import TypeAlias, Any, Union
import einops

NodeType: TypeAlias = Union[tuple[int, dict[str, Any]], int]


In [3]:
# for the simplest case, consider a linear chain with symmetric payoff functions like the quantum prisoners' dilemma.
def get_state(L: int, type: str = "GHZ", dtype: np.dtype = np.complex128):
    if type == "GHZ":
        psi = np.zeros(2**L, dtype=dtype)
        psi[0] = 1/np.sqrt(2)
        psi[-1] = 1/np.sqrt(2)
    elif type == "W":
        psi = np.zeros(2**L, dtype=dtype)
        for i in range(L):
            psi[2**i] = 1/np.sqrt(L)
    elif type == "random":
        # get Haar random state
        psi = np.random.randn(2**L, dtype=dtype) + 1j * np.random.randn(2**L, dtype=dtype)
        psi = psi / np.linalg.norm(psi)
    elif type == "cluster":
        # CZ = np.diag([1, 1, 1, -1])

        # # apply Hadamard gate to each qubit, or initialize to the all plus state
        # psi = reduce(np.kron, 1/np.sqrt(2) * np.array([1, 1]))
        # # apply CZ gate between each pair of qubits
        raise NotImplementedError("Cluster state not yet implemented")
    return psi

# def swap_gate(dtype: np.dtype = np.complex128):
#     return np.array(
#         [[1, 0, 0, 0],
#          [0, 0, 1, 0],
#          [0, 1, 0, 0],
#          [0, 0, 0, 1]],
#         dtype=dtype
#     ).reshape(2, 2, 2, 2)

def get_hamiltonian(L: int, H: Complex[Array, "player cl cr"], edge: tuple[int, int], site: int, dtype: np.dtype = np.complex128) -> Complex[Array, "player Cl Cr"]:
    """
    Get the Hamiltonian for a given set of sites.
    """
    idx = edge.index(site)
    H_mpo = np.eye(2**L, dtype=dtype).reshape([2] * (2*L))
    H_mpo = np.tensordot(H_mpo, H[idx].reshape(2, 2, 2, 2), axes=([L+edge[0], L+edge[1]], [0, 1]))
    H_mpo = np.moveaxis(H_mpo, [-2, -1], [L+edge[0], L+edge[1]])
    return H_mpo

def test_get_hamiltonian():
    H = np.stack([np.diag([3, 0, 5, 1]), np.diag([3, 5, 0, 1])])
    H_mpo = get_hamiltonian(L=3, H=H, edge=(2, 0), site=2) + get_hamiltonian(L=3, H=H, edge=(2, 1), site=2)
    H_matrix_form = einops.rearrange(H_mpo, "a1 a2 a3 b1 b2 b3 -> (a1 a2 a3) (b1 b2 b3)")
    # H_mat_form = H_mpo.reshape(8, 8)
    print(H_matrix_form.real)

def get_hloc_from_graph(G: nx.DiGraph, H: Complex[Array, "player cl cr"], site: int, dtype: np.dtype = np.complex128, normalize: bool = True) -> Complex[Array, "player Cl Cr"]:
    print(G.number_of_nodes())
    hloc = np.zeros([2] * (2 * G.number_of_nodes()), dtype=dtype)
    for edge in G.in_edges(site):
        print(edge)
        H_mpo = get_hamiltonian(L=G.number_of_nodes(), H=H, edge=edge, site=site)
        hloc = hloc + H_mpo
    for edge in G.out_edges(site):
        print(edge)
        H_mpo = get_hamiltonian(L=G.number_of_nodes(), H=H, edge=edge, site=site)
        hloc = hloc + H_mpo

    if normalize:
        hloc = hloc / (len(G.in_edges(site)) + len(G.out_edges(site)))
    return hloc

def test_get_hloc_from_graph():
    game = nx.DiGraph()
    game.add_edge(0, 1)
    game.add_edge(1, 2)
    game.add_edge(0, 2) # NOTE: for prisoner's dilemma, the edge direction does not matter because the payoff functions are related by a swap of qubits.
    print(game.number_of_nodes())
    hloc = get_hloc_from_graph(game, H, 0, normalize=False)
    print(hloc.reshape(8, 8).real)

test_get_hloc_from_graph()


3


NameError: name 'H' is not defined

## Translationally invariant NEs

In [4]:
game = nx.DiGraph()
L = 4
game.add_edges_from([(i, i+1) for i in range(L-1)] + [(L-1, 0)])



## directly optimizing the MPS

In [ ]:
import torch as t
from torch import Tensor

